# 对pay_price大于0的样本进行2分类训练以及stacking分类预测

In [ ]:
import pandas as pd

data_train_ = pd.read_csv("pay_pricedayu0.csv")
# data_train = pd.read_csv("pay_pricedengyu0.csv")
dt1=pd.to_datetime(data_train_["register_time"])
data_train_["register_time"] = dt1.dt.dayofyear
if "prediction_pay_price" in data_train_.columns:
    data_train_ = data_train_.drop("prediction_pay_price",axis=1) 
data_train_1 = data_train_.copy()
data_train_ = data_train_.iloc[:,1:]
data_train_.shape

In [ ]:
import lightgbm as lgb
import numpy as np
from sklearn.model_selection import train_test_split   # cross_validation
from sklearn.linear_model import RidgeClassifierCV,RidgeClassifier,LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,AdaBoostClassifier,BaggingClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA, KernelPCA
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import BernoulliNB,GaussianNB
from sklearn.neighbors import RadiusNeighborsClassifier,NearestCentroid,KNeighborsClassifier
from sklearn.semi_supervised import LabelPropagation,LabelSpreading
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis,LinearDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.base import BaseEstimator, TransformerMixin
import gc
import warnings
warnings.filterwarnings('ignore')
import xgboost as xgb

In [ ]:
class add_feature(BaseEstimator, TransformerMixin):
    def __init__(self, additional=1):
        self.additional = additional

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X["shaman_rate"] = X["shaman_reduce_value"]/(X["shaman_add_value"]+0.1)
        
        return X
    
    
pipe = Pipeline([
        ('add_feature', add_feature(additional=2))
    ])

data_train_ = pipe.fit_transform(data_train_)
data_train_['lable_2'] = data_train_['lable']
del data_train_['lable']
data_train_.rename(columns={'lable_2':'lable'}, inplace = True)

# data_train_.to_csv('tezhengceshi.csv',index=None)
# data_train_['pvp_lanch*pay_count>mean'].sum()

In [ ]:
data_train = data_train_
x, y = data_train.iloc[:, 0:len(data_train.columns)-1], data_train.iloc[:,len(data_train.columns)-1:]
x.shape,y.columns

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1, test_size=0.5)

In [ ]:
all_test=pd.read_csv('test_pay_pricedayu0.csv')
fit_data = all_test.iloc[:,1:108]
dt1=pd.to_datetime(fit_data["register_time"])
fit_data["register_time"] = dt1.dt.dayofyear
fit_data = pipe.fit_transform(fit_data)

In [ ]:
from sklearn.model_selection import StratifiedKFold

def stacking(x_train, y_train, x_test, cv):

#     le = LabelEncoder()
#     y_train = le.fit_transform(y_train)

    clfs = [
        xgb.XGBClassifier(max_depth = 5),
        lgb.LGBMClassifier(max_depth = 5),
        RandomForestClassifier(max_depth = 6),
        GradientBoostingClassifier(max_depth = 5),
    ]

    skf = StratifiedKFold(n_splits=cv)
    skf_dataset = list(skf.split(x_train, y_train))

    # y_count: the kinds of labels
    y_count = len(set(y_train.lable))
#     print y_count
    
    # blend_train is the probabilities that every clf predicts every label (i.e. y) for every sample
    # it is used to train the clfs in the second layer
    blend_train = np.zeros((x_train.shape[0], len(clfs) * y_count))
    blend_train = pd.DataFrame(blend_train)

    # blend_test is used as the input of the clfs in the second layer to predict the labels of x_test
    blend_test = np.zeros((x_test.shape[0], len(clfs) * y_count))
    blend_test = pd.DataFrame(blend_test)

    for j, clf in enumerate(clfs):
        print j, clf

        # blend_test_j: the probabilities that j-th clf predicts every label for x_test
        blend_test_j = np.zeros((x_test.shape[0], len(skf_dataset) * y_count))
        blend_test_j = pd.DataFrame(blend_test_j)
        
        for k, (train_idx, test_idx) in enumerate(skf_dataset):
            x_train_k = x_train.iloc[train_idx,:]
            y_train_k = y_train.iloc[train_idx,:]
            x_train_holdout = x_train.iloc[test_idx,:]

            clf.fit(x_train_k, y_train_k)
            x_train_holdout_ = clf.predict_proba(x_train_holdout)
            
            #blend_train[test_idx, j*y_count: (j+1)*y_count] = clf.predict_proba(x_train_holdout)
            blend_train.iloc[test_idx,j*y_count:(j+1)*y_count] = clf.predict_proba(x_train_holdout)
            blend_test_j.iloc[:, k*y_count: (k+1)*y_count] = clf.predict_proba(x_test)

        # because there are len(skf_dataset) blend_test_j for x_test, it needs to calculated the mean value
        blend_test_j_mean = np.zeros((x_test.shape[0], y_count))
        blend_test_j_mean = pd.DataFrame(blend_test_j_mean)

        # indices: supposed y_count = 3, indices would be [0, 3, 6]
        # it is used to find the corresponding probabilities of the same label, and calculate the mean
        indices = np.arange(len(skf_dataset)) * y_count
        for c in xrange(y_count):
            blend_test_j_mean.iloc[:,c] = pd.DataFrame(blend_test_j.iloc[:, indices].mean(1))
            indices += 1
        blend_test[range(j*y_count,(j+1)*y_count)] = blend_test_j_mean
#         print blend_test_j_mean.head()
#         print blend_test.head()

#     clf = LogisticRegression()
#     clf.fit(blend_train, y_train)
#     y_pred = clf.predict(blend_test)
#     y_pred = le.inverse_transform(y_pred)

    return blend_train,blend_test

In [ ]:
stacking_train,stacking_test = stacking(x,y,fit_data,4)
stacking_train['user_id'] = data_train_1['user_id']
stacking_test['user_id'] = all_test['user_id']
stacking_train['lable'] = data_train_1['lable']
stacking_train.to_csv('stacking_train12.csv',index=None)
stacking_test.to_csv('stacking_test12.csv',index=None)

In [ ]:
stacking_train12 = stacking_train
stacking_test12 = stacking_test
stacking_train12_ = stacking_train12.iloc[:,:8]
stacking_test12_ = stacking_test12.iloc[:,:8]
# stacking_train12_.columns,stacking_test12_.columns
lr = LogisticRegression()
lr.fit(stacking_train12_, stacking_train12.lable)
y_pred = lr.predict(stacking_test12_)
y_pred = pd.DataFrame(y_pred)
stacking_test12['lable'] = y_pred
stacking_test12__ =stacking_test12.iloc[:,[8,9]]

In [ ]:
stacking_test12__['lable'] = stacking_test12__.apply(lambda x: 2 if(x['lable'] == 0) else 1,axis=1)   
stacking_test12__.to_csv('stacking_test12_lr.csv',index=None)